计算`Fama-French`三因子案例涉及到：
1. 如何在`Notebook`运行回测
2. `Fundamentals`及财务数据
3. `pipeline`及自定义因子用法
4. 回测速度

In [1]:
%load_ext zipline

In [2]:
%%zipline --start 2017-1-1 --end 2018-1-1
from zipline.api import symbol, sid, get_datetime

import pandas as pd
import numpy as np
from zipline.api import attach_pipeline, pipeline_output
from zipline.pipeline import Pipeline
from zipline.pipeline import CustomFactor
from zipline.pipeline.data import USEquityPricing
from zipline.pipeline.fundamentals import Fundamentals

# time frame on which we want to compute Fama-French
normal_days = 31
# approximate the number of trading days in that period
# this is the number of trading days we'll look back on,
# on every trading day.
business_days = int(0.69 * normal_days)

# 以下自定义因子选取期初数
class Returns(CustomFactor):
    """
    每个交易日每个股票窗口长度"business_days"期间收益率
    """
    window_length = business_days
    inputs = [USEquityPricing.close]
    def compute(self, today, assets, out, price):
        out[:] = (price[-1] - price[0]) / price[0] * 100

class MarketEquity(CustomFactor):
    """
    每个交易日每只股票所对应的总市值
    """
    window_length = business_days
    inputs = [USEquityPricing.tmv]
    def compute(self,today,assets,out,mcap):
        out[:] = mcap[0]

class BookEquity(CustomFactor):
    """
    每个交易日每只股票所对应的账面价值（所有者权益）
    """
    window_length = business_days
    inputs = [Fundamentals.balance_sheet.A107]
    def compute(self, today, assets, out, book):
        out[:] = book[0]
                                                                             
def initialize(context):
    """
    use our factors to add our pipes and screens.
    """
    pipe = Pipeline()
    mkt_cap = MarketEquity()
    pipe.add(mkt_cap,'market_cap')
    
    book_equity = BookEquity()
    # book equity over market equity
    be_me = book_equity / mkt_cap
    pipe.add(be_me,'be_me')

    returns = Returns()
    pipe.add(returns,'returns')

    attach_pipeline(pipe, 'ff_example')

    
def before_trading_start(context,data):
    """
    every trading day, we use our pipes to construct the Fama-French
    portfolios, and then calculate the Fama-French factors appropriately.
    """
     
    factors = pipeline_output('ff_example')
    
    # get the data we're going to use
    returns = factors['returns']
    mkt_cap = factors.sort_values(['market_cap'], ascending=True)
    be_me = factors.sort_values(['be_me'], ascending=True)
    
    # to compose the six portfolios, split our universe into portions
    half = int(len(mkt_cap) * 0.5)
    small_caps = mkt_cap[:half]
    big_caps = mkt_cap[half:]
    
    thirty = int(len(be_me) * 0.3)
    seventy = int(len(be_me) * 0.7)
    growth = be_me[:thirty]
    neutral = be_me[thirty:seventy]
    value = be_me[seventy:]
    
    # now use the portions to construct the portfolios.
    # note: these portfolios are just lists (indices) of equities
    small_value = small_caps.index.intersection(value.index)
    small_neutral = small_caps.index.intersection(neutral.index)
    small_growth = small_caps.index.intersection(growth.index)
    
    big_value = big_caps.index.intersection(value.index)
    big_neutral = big_caps.index.intersection(neutral.index)
    big_growth = big_caps.index.intersection(growth.index)
    
    # take the mean to get the portfolio return, assuming uniform
    # allocation to its constituent equities.
    sv = returns[small_value].mean()
    sn = returns[small_neutral].mean()
    sg = returns[small_growth].mean()
    
    bv = returns[big_value].mean()
    bn = returns[big_neutral].mean()
    bg = returns[big_growth].mean()
    
    # computing Rm-Rf (Market Returns - Risk-Free Returns).  we take the
    # rate of risk-free returns to be zero, so this is simply SPY's returns.
    # have to set an initial dummy value
    #context.rm_rf = float('nan')
    #if spy in returns.index:
        #context.rm_rf = returns.loc[spy]
    
    # computing SMB
    context.smb = (sv + sn + sg) / 3 - (bv + bn + bg) / 3
    
    # computing HML
    context.hml = (sv + bv) / 2 - (sg + bg) / 2

def handle_data(context, data):
    # print the Fama-French factors for the period defined by business_days
    # ending on the previous trading day.
    print(context.smb, context.hml)

4.72746207088731 4.171923693774259
4.7180450348714755 4.551002578333151
4.364296306063149 4.570034537074281
4.0801429884957106 4.284954342431844
4.665688377507024 5.4911184160784785
4.798909253374471 5.650450474461904
6.227081283327899 4.0943179643173435
6.145113250509516 4.550714772106331
5.42677820521226 4.23550480729464
3.7193394615635293 4.7527891497408365
0.8846901571705743 6.226379567053928
0.7972766723829343 5.305831945241825
0.6469930712466869 6.8870581547175
0.3961900505650302 6.90029319905843
0.6306789244429227 6.442489935283322
1.381271677916877 6.732876645378775
0.6725505758664192 6.752043465834369
0.014014289806335789 6.605843892342312
-0.11424607900759165 6.955457506878365
0.2459618864708797 7.063820418114889
0.8937587571785575 7.024864392053409
0.742890752779819 6.420461703374208
1.3529261672137676 7.164880004150782
1.86383321086846 7.601385395774351
1.2292438296263537 7.611612173056685
1.0883221343630576 7.014362272695433
1.3592451939601462 8.071930841374543
1.317409063

-3.298409416729659 1.3322111006044768
-3.3901858630081634 1.832270898103097
-2.61085015027773 2.7072777859272534
-2.7134045954101813 2.890994128047682
-2.965097825507776 4.4434701009908615
-2.127286336673744 4.82239123365467
-2.333702398605596 4.586073063874492
-3.317213538297726 5.777677411400976
-3.7826490774625467 5.865417539360356
-3.968050312886475 4.632130457644143
-3.6049122660042263 4.272829473636126
-2.826203983830614 5.1387725284530745
-2.7521406800197976 5.073758872417522
-3.122965381291407 4.053221959686246
-3.63634992026587 3.1695547361855194
-3.082042587197278 2.944074280629926
-1.27665563983915 1.5563863738935608
-0.839094506478999 2.964185980308965
-0.06843972383450803 2.7904394389547664
-0.46753918103098613 2.118193753236361
-1.345017217265104 0.8349292911638904
-1.4512950168639096 0.7013792195575688
-1.8721937709387417 -0.035943178624163075
-1.3512470071231586 -0.008890430612974676
-0.6998161264349445 -0.29733135321890636
-1.1942883365086068 -0.38062423581176485
[2018

,algo_volatility,algorithm_period_return,alpha,benchmark_period_return,benchmark_volatility,beta,capital_used,ending_cash,ending_exposure,ending_value,...,short_exposure,short_value,shorts_count,sortino,starting_cash,starting_exposure,starting_value,trading_days,transactions,treasury_period_return
2017-01-03 07:00:00+00:00,NaN,0.0,NaN,0.009712,NaN,NaN,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,1,[],0.0
2017-01-04 07:00:00+00:00,0.0,0.0,0.0,0.017593,0.021406,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,2,[],0.0
2017-01-05 07:00:00+00:00,0.0,0.0,0.0,0.017435,0.083084,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,3,[],0.0
2017-01-06 07:00:00+00:00,0.0,0.0,0.0,0.011356,0.115404,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,4,[],0.0
2017-01-09 07:00:00+00:00,0.0,0.0,0.0,0.016261,0.100950,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,5,[],0.0
2017-01-10 07:00:00+00:00,0.0,0.0,0.0,0.014560,0.095760,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,6,[],0.0
2017-01-11 07:00:00+00:00,0.0,0.0,0.0,0.007377,0.104382,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,7,[],0.0
2017-01-12 07:00:00+00:00,0.0,0.0,0.0,0.002279,0.102578,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,8,[],0.0
2017-01-13 07:00:00+00:00,0.0,0.0,0.0,0.002971,0.095975,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,9,[],0.0
2017-01-16 07:00:00+00:00,0.0,0.0,0.0,0.002830,0.090519,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,10,[],0.0


**耗时11.5秒**